In [1]:
import json
import os
import numpy as np
from sklearn.metrics import label_ranking_average_precision_score,average_precision_score,precision_recall_curve

In [22]:
with open(os.path.join('test_results','test_result_part1.json'),'r') as f:
    results = json.load(f)

In [48]:
results.keys()

dict_keys(['/home/tomita/keras-yolo3/use_images/n02231487/n02231487_2251.jpg', '/home/tomita/keras-yolo3/use_images/n02231487/n02231487_143.jpg', '/home/tomita/keras-yolo3/use_images/n02231487/n02231487_1788.jpg', '/home/tomita/keras-yolo3/use_images/n02231487/n02231487_8823.jpg', '/home/tomita/keras-yolo3/use_images/n02231487/n02231487_6784.jpg', '/home/tomita/keras-yolo3/use_images/n02231487/n02231487_9227.jpg', '/home/tomita/keras-yolo3/use_images/n02231487/n02231487_4525.jpg', '/home/tomita/keras-yolo3/use_images/n02231487/n02231487_4030.jpg', '/home/tomita/keras-yolo3/use_images/n02231487/n02231487_1002.jpg', '/home/tomita/keras-yolo3/use_images/n02231487/n02231487_9508.jpg', '/home/tomita/keras-yolo3/use_images/n02231487/n02231487_10829.jpg', '/home/tomita/keras-yolo3/use_images/n02231487/n02231487_5700.jpg', '/home/tomita/keras-yolo3/use_images/n02231487/n02231487_7500.jpg', '/home/tomita/keras-yolo3/use_images/n02231487/n02231487_10506.jpg', '/home/tomita/keras-yolo3/use_images

In [4]:
partial = results['/home/tomita/keras-yolo3/use_images/n02231487/n02231487_143.jpg']

In [5]:
partial_true = partial["ground truth"]
partial_predict = partial["prediction"]

In [3]:
def bb_intersection_over_union(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    # compute the area of intersection rectangle
    interArea = abs(max((xB - xA, 0)) * max((yB - yA), 0))
    if interArea == 0:
        return 0
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = abs((boxA[2] - boxA[0]) * (boxA[3] - boxA[1]))
    boxBArea = abs((boxB[2] - boxB[0]) * (boxB[3] - boxB[1]))

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)

    # return the intersection over union value
    return iou

def mean_average_precision(y_true, y_pred):
    average_precisions = []
    # クラス単位でAPを計算
    for i in range(len(y_true)):
        sort_idx = np.argsort(y_pred[i])[::-1]
        y_true_sorted = y_true[i][sort_idx]

        cumsum = np.cumsum(y_true_sorted)
        recall = cumsum / np.max(cumsum)
        precision = cumsum / np.arange(1, 1 + y_true[i].shape[0])

        # 代表点
        points_x = np.arange(11) / 10
        points_y = np.zeros(points_x.shape[0])
        for i in range(points_x.shape[0]-1, -1, -1):
            points_y[i] = np.max(precision[recall >= points_x[i]])

        average_precision = np.mean(points_y)
        average_precisions.append(average_precision)
    return average_precisions

In [7]:
true_bbox = []
for t in partial_true:
    true_bbox.append(
        [
            t["bbox"]["left"],
            t["bbox"]["top"],
            t["bbox"]["right"],
            t["bbox"]["bottom"]
        ]
    )

predict_bbox = []
predict_score = []
for t in partial_predict:
    predict_bbox.append(
        [
            t["bbox"]["left"],
            t["bbox"]["top"],
            t["bbox"]["right"],
            t["bbox"]["bottom"],
        ]
    )
    predict_score.append(t["score"])

In [8]:
true_bbox

[[184, 168, 248, 242]]

In [9]:
predict_bbox

[[275, 31, 399, 199], [158, 64, 388, 264]]

In [23]:
AP = {}
for i,target in enumerate(results.keys()):
    print(i,target)
    partial = results[target]
    partial_true = partial["ground truth"]
    partial_predict = partial["prediction"]
    true_bbox = []
    for t in partial_true:
        true_bbox.append(
            [
                t["bbox"]["left"],
                t["bbox"]["top"],
                t["bbox"]["right"],
                t["bbox"]["bottom"]
            ]
        )

    predict_bbox = []
    predict_score = []
    for t in partial_predict:
        predict_bbox.append(
            [
                t["bbox"]["left"],
                t["bbox"]["top"],
                t["bbox"]["right"],
                t["bbox"]["bottom"],
            ]
        )
        predict_score.append(t["score"])
    
    iou = []
    score = []
    for tb in true_bbox:
        for j,pb in enumerate(predict_bbox):
            print("iou:",bb_intersection_over_union(tb,pb),"score:",predict_score[j])
            iou.append(bb_intersection_over_union(tb,pb))
            score.append(predict_score[j])
    iou = np.array(iou)
    iou = np.where(iou > 0.25, True, False)
    score = np.array(score)
    precision, recall, threshold = precision_recall_curve(iou[::-1],score[::-1])
    print("precision:",precision)
    print("recall:",recall)
    print("threshold:",threshold)
    #print(iou,score)
    #print(average_precision_score(iou,score))
    AP[target] = average_precision_score(iou,score)

0 /home/tomita/keras-yolo3/use_images/n02231487/n02231487_2251.jpg
iou: 0.6654664167043765 score: 0.07124502956867218
iou: 0.5019975981727275 score: 0.28216853737831116
precision: [1. 1. 1.]
recall: [1.  0.5 0. ]
threshold: [0.07124503 0.28216854]
1 /home/tomita/keras-yolo3/use_images/n02231487/n02231487_143.jpg
iou: 0 score: 0.053225111216306686
iou: 0.10295652173913043 score: 0.19599822163581848
precision: [0. 1.]
recall: [0. 0.]
threshold: [0.19599822]
2 /home/tomita/keras-yolo3/use_images/n02231487/n02231487_1788.jpg
iou: 0 score: 0.14917005598545074
iou: 0.04659904163295787 score: 0.32592394948005676
iou: 0.13342862469703562 score: 0.32779866456985474
precision: [0. 1.]
recall: [0. 0.]
threshold: [0.32779866]
3 /home/tomita/keras-yolo3/use_images/n02231487/n02231487_8823.jpg
iou: 0.7504558106941949 score: 0.05859237536787987
iou: 0.35250977835723596 score: 0.12749522924423218
iou: 0.4158210692136298 score: 0.7610382437705994
precision: [1. 1. 1. 1.]
recall: [1.         0.66666667 

recall: [1.  0.5 0. ]
threshold: [0.30053961 0.39464748]
34 /home/tomita/keras-yolo3/use_images/n02231487/n02231487_9254.jpg
iou: 0.09017656950672645 score: 0.10791382193565369
precision: [0. 1.]
recall: [0. 0.]
threshold: [0.10791382]
35 /home/tomita/keras-yolo3/use_images/n02231487/n02231487_189.jpg
iou: 0.5045907191349873 score: 0.05152406170964241
iou: 0.22596874200582281 score: 0.11335223913192749
iou: 0.3937632950026205 score: 0.3061662018299103
iou: 0.3637868426769845 score: 0.6418434381484985
iou: 0 score: 0.05152406170964241
iou: 0 score: 0.11335223913192749
iou: 0.2863468418274588 score: 0.3061662018299103
iou: 0.2008554956295332 score: 0.6418434381484985
precision: [0.5  0.5  0.75 0.5  1.  ]
recall: [1.   0.75 0.75 0.25 0.  ]
threshold: [0.05152406 0.11335224 0.3061662  0.64184344]
36 /home/tomita/keras-yolo3/use_images/n02231487/n02231487_429.jpg
iou: 0.49154239263620336 score: 0.22819313406944275
precision: [1. 1.]
recall: [1. 0.]
threshold: [0.22819313]
37 /home/tomita/ke

In [24]:
AP

{'/home/tomita/keras-yolo3/use_images/n02231487/n02231487_1002.jpg': 1.0,
 '/home/tomita/keras-yolo3/use_images/n02231487/n02231487_10120.jpg': -0.0,
 '/home/tomita/keras-yolo3/use_images/n02231487/n02231487_10201.jpg': 0.5833333333333333,
 '/home/tomita/keras-yolo3/use_images/n02231487/n02231487_10352.jpg': -0.0,
 '/home/tomita/keras-yolo3/use_images/n02231487/n02231487_10371.jpg': 1.0,
 '/home/tomita/keras-yolo3/use_images/n02231487/n02231487_10499.jpg': 1.0,
 '/home/tomita/keras-yolo3/use_images/n02231487/n02231487_10506.jpg': 1.0,
 '/home/tomita/keras-yolo3/use_images/n02231487/n02231487_10829.jpg': 1.0,
 '/home/tomita/keras-yolo3/use_images/n02231487/n02231487_10982.jpg': 0.5833333333333333,
 '/home/tomita/keras-yolo3/use_images/n02231487/n02231487_1100.jpg': -0.0,
 '/home/tomita/keras-yolo3/use_images/n02231487/n02231487_11404.jpg': 1.0,
 '/home/tomita/keras-yolo3/use_images/n02231487/n02231487_12685.jpg': 1.0,
 '/home/tomita/keras-yolo3/use_images/n02231487/n02231487_12911.jpg':

In [21]:
AP

{'/home/tomita/keras-yolo3/use_images/n02231487/n02231487_1002.jpg': 0.8333333333333333,
 '/home/tomita/keras-yolo3/use_images/n02231487/n02231487_10120.jpg': -0.0,
 '/home/tomita/keras-yolo3/use_images/n02231487/n02231487_10201.jpg': 0.8333333333333333,
 '/home/tomita/keras-yolo3/use_images/n02231487/n02231487_10352.jpg': -0.0,
 '/home/tomita/keras-yolo3/use_images/n02231487/n02231487_10371.jpg': 0.9166666666666665,
 '/home/tomita/keras-yolo3/use_images/n02231487/n02231487_10499.jpg': 1.0,
 '/home/tomita/keras-yolo3/use_images/n02231487/n02231487_10506.jpg': 0.6388888888888888,
 '/home/tomita/keras-yolo3/use_images/n02231487/n02231487_10829.jpg': 1.0,
 '/home/tomita/keras-yolo3/use_images/n02231487/n02231487_10982.jpg': -0.0,
 '/home/tomita/keras-yolo3/use_images/n02231487/n02231487_1100.jpg': -0.0,
 '/home/tomita/keras-yolo3/use_images/n02231487/n02231487_11404.jpg': 1.0,
 '/home/tomita/keras-yolo3/use_images/n02231487/n02231487_12685.jpg': 1.0,
 '/home/tomita/keras-yolo3/use_images/

In [15]:
precision_recall_curve(np.array([0, 0, 1, 1]),np.array([0.1, 0.4, 0.35, 0.8]))

(array([0.66666667, 0.5       , 1.        , 1.        ]),
 array([1. , 0.5, 0.5, 0. ]),
 array([0.35, 0.4 , 0.8 ]))

In [8]:
precision

array([1., 1.])

In [9]:
recall

array([1., 0.])

In [10]:
threshold

array([0.33304778])

In [17]:
AP

{'/home/tomita/keras-yolo3/use_images/n02231487/n02231487_1002.jpg': 1.0,
 '/home/tomita/keras-yolo3/use_images/n02231487/n02231487_10120.jpg': 1.0,
 '/home/tomita/keras-yolo3/use_images/n02231487/n02231487_10201.jpg': 1.0,
 '/home/tomita/keras-yolo3/use_images/n02231487/n02231487_10352.jpg': 1.0,
 '/home/tomita/keras-yolo3/use_images/n02231487/n02231487_10371.jpg': 1.0,
 '/home/tomita/keras-yolo3/use_images/n02231487/n02231487_10499.jpg': 1.0,
 '/home/tomita/keras-yolo3/use_images/n02231487/n02231487_10506.jpg': 1.0,
 '/home/tomita/keras-yolo3/use_images/n02231487/n02231487_10829.jpg': 1.0,
 '/home/tomita/keras-yolo3/use_images/n02231487/n02231487_10982.jpg': 1.0,
 '/home/tomita/keras-yolo3/use_images/n02231487/n02231487_1100.jpg': 1.0,
 '/home/tomita/keras-yolo3/use_images/n02231487/n02231487_11404.jpg': 1.0,
 '/home/tomita/keras-yolo3/use_images/n02231487/n02231487_12685.jpg': 1.0,
 '/home/tomita/keras-yolo3/use_images/n02231487/n02231487_12911.jpg': 1.0,
 '/home/tomita/keras-yolo3/

In [112]:
arr

array([[0.50459072, 0.22596874, 0.3937633 , 0.36378684, 0.        ,
        0.        , 0.28634684, 0.2008555 ],
       [0.05152406, 0.11335224, 0.3061662 , 0.64184344, 0.05152406,
        0.11335224, 0.3061662 , 0.64184344]])

In [16]:
arr[0] = np.where(arr[0] > 0.2, True, False)

In [17]:
arr

array([[1.        , 1.        , 1.        , 1.        , 0.        ,
        0.        , 1.        , 1.        ],
       [0.05152406, 0.11335224, 0.3061662 , 0.64184344, 0.05152406,
        0.11335224, 0.3061662 , 0.64184344]])

In [120]:
arr[0][::-1]

array([1., 1., 0., 0., 1., 1., 1., 1.])

In [115]:
AP = 0
for a in arr.T:
    print(a[0] * a[1])
    AP += a[0] * a[1]

0.05152406170964241
0.11335223913192749
0.3061662018299103
0.6418434381484985
0.0
0.0
0.3061662018299103
0.6418434381484985


In [116]:
AP

2.0608955807983875

In [117]:
AP/len(arr.T)

0.25761194759979844

In [20]:
mean_average_precision([arr[0]],[arr[1]])

[0.9242424242424243]

In [37]:
label_ranking_average_precision_score([arr[0][::-1]],[arr[1][::-1]])

0.9305555555555555

In [38]:
arr

array([[1.        , 1.        , 1.        , 1.        , 0.        ,
        0.        , 1.        , 1.        ],
       [0.05152406, 0.11335224, 0.3061662 , 0.64184344, 0.05152406,
        0.11335224, 0.3061662 , 0.64184344]])

In [28]:
average_precision_score(arr[0][::-1],arr[1][::-1])

0.9305555555555556

In [25]:
average_precision_score(np.array([0, 0, 1, 1]),np.array([0.1, 0.4, 0.35, 0.8]))

0.8333333333333333

In [26]:
np.array([0.1, 0.4, 0.35, 0.8])[::-1]

array([0.8 , 0.35, 0.4 , 0.1 ])